<a href="https://colab.research.google.com/github/OnlyBelter/machine-learning-note/blob/master/GNN/GNN_basic_concept.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch-geometric

# 1. 定义图结构和初始节点特征

In [2]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

# 定义图结构 - 边列表表示法
edge_index = torch.tensor([
    [0, 1, 0, 4, 1, 2, 1, 3, 3, 4],  # 源节点
    [1, 0, 4, 0, 2, 1, 3, 1, 4, 3]   # 目标节点
], dtype=torch.long)  # 无向图需要双向边

# 初始节点特征 (每个节点有3维特征)
x = torch.tensor([
    [1, 0, 0],  # 节点0的特征
    [0, 1, 0],  # 节点1的特征
    [0, 0, 1],  # 节点2的特征
    [1, 1, 0],  # 节点3的特征
    [0, 1, 1]   # 节点4的特征
], dtype=torch.float)


这部分定义了我们的5节点图。edge_index使用COO格式表示边，第一行是源节点，第二行是目标节点。x是每个节点的初始特征，这里我们使用了3维特征。

# 2. 定义GCN模型

In [3]:
class SimpleGNN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimpleGNN, self).__init__()
        # 第一层GCN: 输入维度，输出维度
        self.conv1 = GCNConv(input_dim, hidden_dim)
        # 第二层GCN: 输入维度，输出维度
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        # 第一层卷积 + 激活函数
        h = self.conv1(x, edge_index)
        h = F.relu(h)

        # 第二层卷积
        h = self.conv2(h, edge_index)

        return h  # 返回所有节点的最终表示


这是一个简单的两层GCN模型。第一层将节点特征从input_dim维映射到hidden_dim维，然后应用ReLU激活函数。第二层将特征进一步映射到output_dim维，得到最终的节点表示。

# 3. 初始化模型

In [4]:
input_dim = 3  # 初始节点特征维度
hidden_dim = 8  # 隐藏层维度
output_dim = 16  # 输出节点表示维度

model = SimpleGNN(input_dim, hidden_dim, output_dim)


# 4. 获取节点表示

In [5]:
model.eval()
with torch.no_grad():
    node_embeddings = model(x, edge_index)


将模型设置为评估模式，然后进行前向传播，得到每个节点的表示。

# 5. 打印节点表示

In [6]:
print("节点表示向量：")
for i, embedding in enumerate(node_embeddings):
    print(f"节点 {i} 的表示: {embedding}")


节点表示向量：
节点 0 的表示: tensor([ 0.0392,  0.2471,  0.0344, -0.3071,  0.0045,  0.3343, -0.1731, -0.3266,
        -0.5102,  0.0605,  0.3149,  0.0173,  0.1235, -0.4152, -0.1310, -0.0107])
节点 1 的表示: tensor([ 0.0662,  0.2285,  0.0284, -0.4020, -0.0368,  0.3530, -0.2147, -0.4198,
        -0.6632,  0.0690,  0.4349, -0.0095,  0.1799, -0.5193, -0.1640, -0.0150])
节点 2 的表示: tensor([ 0.0409,  0.1648,  0.0528, -0.2432, -0.0502,  0.2052, -0.1131, -0.2292,
        -0.3870,  0.0266,  0.2469, -0.0068,  0.1483, -0.3437, -0.1078, -0.0562])
节点 3 的表示: tensor([ 0.0505,  0.2498,  0.0177, -0.3534,  0.0148,  0.3712, -0.2052, -0.3898,
        -0.5958,  0.0731,  0.3736,  0.0117,  0.1270, -0.4523, -0.1443,  0.0152])
节点 4 的表示: tensor([ 0.0572,  0.2326,  0.0045, -0.3786,  0.0089,  0.3747, -0.2208, -0.4207,
        -0.6397,  0.0801,  0.4119,  0.0041,  0.1304, -0.4716, -0.1503,  0.0272])


# 6. 手动跟踪GCN计算过程

In [7]:
def manual_gcn_layer(x, edge_index, weight):
    """手动实现一个GCN层的计算过程"""
    # 线性变换
    x_transformed = x @ weight

    # 提取源节点和目标节点
    row, col = edge_index

    # 计算度和归一化系数...

    # 消息传递
    out = torch.zeros_like(x_transformed)
    for i in range(edge_index.size(1)):
        src, dst = edge_index[0, i], edge_index[1, i]
        out[dst] += norm[i] * x_transformed[src]

    return out


这个函数手动实现了一个GCN层的计算过程，帮助理解GCN的内部工作原理。它包括线性变换、计算归一化系数和消息传递三个步骤。